## imports

In [1]:
import os
from geopandas import read_file
from geojson import Feature, FeatureCollection, dump
import json
import simplekml
from pyproj import transform, Proj

data_partition = 'd:\\data\\resowrs'

hexgrid_dir = os.path.join(data_partition, 'hexgrid')
base_file_path = os.path.join(hexgrid_dir, 'grids', 'hex_basegrid.geojson')
out_file_path = os.path.join(hexgrid_dir, 'hexgrid.geojson')

base_grid = read_file(base_file_path)

crs = {
        "type": "name",
        "properties": {"name": base_grid.crs['init']}
      }

def WKT(hexgrid_df):
    
    west = hexgrid_df['west'].values[0]
    east = hexgrid_df['east'].values[0]
    north = hexgrid_df['north'].values[0]
    south = hexgrid_df['south'].values[0]
    
    print(f'POLYGON(({west} {north}, {west} {south} ,' + \
           f'{east} {south}, {east} {north}, {west} {north}))')

## run the code

In [2]:
features = []
for index, row in base_grid.iterrows():
    features.append(Feature(geometry=row['geometry'],
                            properties={'id': index}))

feature_collection = FeatureCollection(features, crs=crs)

with open(out_file_path, 'w') as f:
   dump(feature_collection, f)

In [4]:
for cell in [324, 325, 389]:
    
    hexcell_file_path = os.path.join(hexgrid_dir, f'hexgrid_{str(cell)}.geojson')

    features = []
    for index, row in base_grid.iterrows():
        
        if index == cell:
            features.append(Feature(geometry=row['geometry'],
                            properties={'id': index,
                                        'west': row['left'],
                                        'east': row['right'],
                                        'south': row['bottom'],
                                        'north': row['top']}))
            print(row['geometry'])
            
    feature_collection = FeatureCollection(features, crs=crs)

    with open(hexcell_file_path, 'w') as f:
       dump(feature_collection, f)
    
    WKT(read_file(hexcell_file_path))
    print()

MULTIPOLYGON (((-2.6793451 50.4878638, -2.5968334 50.3983859, -2.4343753 50.3991067, -2.353813 50.4893101, -2.4360247 50.5789604, -2.5991015 50.578235, -2.6793451 50.4878638)))
POLYGON((351902.89601377555 75561.165, 351902.89601377555 55561.165 ,374996.9067813606 55561.165, 374996.9067813606 75561.165, 351902.89601377555 75561.165))

MULTIPOLYGON (((-2.5435058 59.3851676, -2.440715 59.295755, -2.237995 59.2962846, -2.1369961 59.38623, -2.2392619 59.4758808, -2.443056 59.475348, -2.5435058 59.3851676)))
POLYGON((369223.40408946434 1065561.165, 369223.40408946434 1045561.165 ,392317.4148570494 1045561.165, 392317.4148570494 1065561.165, 369223.40408946434 1065561.165))

MULTIPOLYGON (((-0.4734599 50.7496472, -0.6370873 50.751673, -0.716448 50.8425047, -0.6318393 50.9314746, -0.4675814 50.9294351, -0.3885631 50.8384398, -0.4734599 50.7496472)))
POLYGON((490466.96061928576 115561.165, 490466.96061928576 95561.165 ,513560.97138687083 95561.165, 513560.97138687083 115561.165, 490466.96061928

In [ ]:
with open(base_file_path) as f:
    data = json.load(f)


projectionIn = Proj(init='epsg:32627')

kml = simplekml.Kml()
for feature in data['features']:
    coord_tuples = []
    coord_list = (feature['geometry']['coordinates'][0][0])
    for coord in coord_list:
        long_w, lat_n = projectionIn(coord[0], coord[1], inverse=True)
#        long_w, lat_n = transform(projectionIn, projectionIn,
#                                  coord[0], coord[1])
        coord_tuples.append((long_w, lat_n))
    print(coord_tuples)
    name = str((int(feature['properties']['id'])))
    if feature['geometry']['type'] == 'MultiPolygon':
        kml.newpolygon(name=name,
                       description='test',
                       outerboundaryis=coord_tuples)
kml.save(base_file_path.replace('geojson', 'kml'))